SALVANDO OS FRAMES DE CADA VIDEO EM UMA SUBPASTA, EXTRAI O NOME DO PRIMEIRO FRAME E NOMEIA A SUBPASTA.

In [ ]:
import os
import re
import cv2
import shutil
import pytesseract
import numpy as np
from PIL import Image

# Caminho do executável da bibiliteca Tesseract (OCR)
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Caminhos das pastas de entrada (frames) e saída (subpastas nomeadas)
entrada = r"D:\Ligia Puc\ProjetoV - A3\framesA3"
saida = r"D:\Ligia Puc\ProjetoV - A3\Base A3"

# Função que extrai e trata a palavra extraída.
def extrair_palavra_da_imagem(caminho_imagem):
    imagem = Image.open(caminho_imagem)
    largura, altura = imagem.size

    # Corta a imagem na área onde o texto aparece.
    esquerda = 0
    topo = 0
    direita = int(largura * 0.39)
    inferior = int(altura * 0.4)
    area_texto = imagem.crop((esquerda, topo, direita, inferior))

    # Extração do texto da imagem cortada
    texto = pytesseract.image_to_string(area_texto, lang='por')

    # Limpa o texto, fazendo os ajustes necessários.
    texto = texto.replace('#', '')
    texto = re.sub(r'\d+', '', texto)
    texto = texto.strip().replace(' ', '_')
    texto = texto.replace('\n', '_').replace('\r', '')  
    texto = re.sub(r'[\\/:*?"<>|]', '', texto) 
    
    return texto if texto else "sem_texto"  

# Lista todas as subpastas dentro da pasta de entrada
subpastas = [d for d in os.listdir(entrada) if os.path.isdir(os.path.join(entrada, d))]
total_subpastas = len(subpastas)
copiados_com_sucesso = 0
falhas = []

# Processa cada subpasta
for idx, subpasta in enumerate(subpastas, 1):
    caminho_subpasta = os.path.join(entrada, subpasta)
    arquivos = sorted(os.listdir(caminho_subpasta))  # Ordena os arquivos da subpasta
    
    if not arquivos:
        print(f"A {subpasta} está vazia.")
        falhas.append(subpasta)
        continue

    # Usa a primeira imagem da subpasta para extrair o texto (nome do sinal).
    primeira_imagem = os.path.join(caminho_subpasta, arquivos[0])
    palavra = extrair_palavra_da_imagem(primeira_imagem)

    if palavra:
        nova_subpasta = os.path.join(saida, palavra)  # Define o novo nome da pasta de destino
        os.makedirs(nova_subpasta, exist_ok=True)

        sucesso = True
        for arquivo in arquivos:
            origem = os.path.join(caminho_subpasta, arquivo)
            destino = os.path.join(nova_subpasta, arquivo)
            try:
                shutil.copy2(origem, destino)  # Copiando cada arquivo para a nova pasta
            except Exception as e:
                print(f"Falha ao copiar {arquivo} de {subpasta}: {e}")
                sucesso = False
                break

        if sucesso:
            copiados_com_sucesso += 1
            print(f"[{idx}/{total_subpastas}] Copiado: {subpasta} para {palavra}")
        else:
            falhas.append(subpasta)
    else:
        print(f"[{idx}/{total_subpastas}] Palavra não encontrada em {subpasta}")
        falhas.append(subpasta)

# Resumo final da execução
print("\n Relatório Final:")
print(f" Subpastas copiadas com sucesso: {copiados_com_sucesso}/{total_subpastas}")
if falhas:
    print(f" Subpastas com falha ({len(falhas)}):")
    for f in falhas:
        print(f"  - {f}")
else:
    print("Todas as subpastas foram copiadas corretamente!")



REMOVENDO PREFIXOS 

In [ ]:

caminho_raiz = r"D:\Ligia Puc\ProjetoV - A3\Base A3"

# Prefixos a serem removidos
prefixos_remover = ['nNn_', 'O_', '_', 'nN_', 'Nn_', 'N_', 'n_', 'ªª_', 'ªª_', '.ª=_',' ª_', '(', ')', '_', 'ªl_', 'ª', '+', 'º_', 'ªl ', '—', '[)__', '!s()', '[)_']

# Renomeia as pastas
for nome in os.listdir(caminho_raiz):
    caminho_antigo = os.path.join(caminho_raiz, nome)

    if os.path.isdir(caminho_antigo):
        novo_nome = nome

        # Remove prefixo
        for prefixo in prefixos_remover:
            if novo_nome.startswith(prefixo):
                novo_nome = novo_nome[len(prefixo):]
                break

        # Se nome ficou vazio
        novo_nome = novo_nome.strip()
        if not novo_nome:
            novo_nome = "sem nome"

        caminho_novo = os.path.join(caminho_raiz, novo_nome)

        if novo_nome != nome:
            if not os.path.exists(caminho_novo):
                os.rename(caminho_antigo, caminho_novo)
                print(f"Renomeado: {nome} para {novo_nome}")
            else:
                print(f"Já existe: {novo_nome} (não renomeado)")


REMOVENDO SULFIXOS 

In [ ]:
caminho_raiz = r"D:\Ligia Puc\ProjetoV - A3\Base A3"

# Sufixos que você quer remover
sufixos_remover = ['_I', '_', '!', '_«', '_—', '_à', '_”', ')', '(', '_Y', '__ªª__', '__.AW', '__P”_ªu', '__&', '__Y', '__—_u_V', '__P_ªª_', '__ªª', '_AJ' ]  # exemplo

# Renomeia as pastas
for nome in os.listdir(caminho_raiz):
    caminho_antigo = os.path.join(caminho_raiz, nome)

    if os.path.isdir(caminho_antigo):
        novo_nome = nome

        # Remove sufixo
        for sufixo in sufixos_remover:
            if novo_nome.endswith(sufixo):
                novo_nome = novo_nome[:-len(sufixo)]
                break

        # Se nome ficou vazio
        novo_nome = novo_nome.strip()
        if not novo_nome:
            novo_nome = "sem nome"

        caminho_novo = os.path.join(caminho_raiz, novo_nome)

        if novo_nome != nome:
            if not os.path.exists(caminho_novo):
                os.rename(caminho_antigo, caminho_novo)
                print(f"Renomeado: {nome} para {novo_nome}")
            else:
                print(f"Já existe: {novo_nome} (não renomeado)")

REMOVENDO ACENTOS E ESPAÇOS DOS NOMES DA PASTAS CRIADAS ACIMA

In [ ]:
import unicodedata

# Pasta contendo as subpastas originais.
origem = r"D:\Ligia Puc\ProjetoV - A3\Base A3"

# Pasta criada para salvar as subpastas renomeadas, sem alterar as originais.
destino = r"D:\Ligia Puc\ProjetoV - A3\BaseA3" 
os.makedirs(destino, exist_ok=True)

# Função para normalizar nomes de pasta
def normalizar_nome(nome):
    # Remove acentos
    nome = unicodedata.normalize('NFD', nome)
    nome = nome.encode('ascii', 'ignore').decode('utf-8')

    # Substituindo ocorrências específicas
    nome = nome.replace(',-', '_').replace(',_', '_')
    nome = nome.replace(', ', '_').replace(' ', '_')
    nome = nome.replace('-', '_')
    nome = re.sub(r'_+', '_', nome)

    # Remove "_" do início ou fim
    nome = nome.strip('_')

    return nome

for pasta in os.listdir(origem):
    caminho_origem = os.path.join(origem, pasta)
    if os.path.isdir(caminho_origem):
        nome_modificado = normalizar_nome(pasta)
        caminho_destino = os.path.join(destino, nome_modificado)

        # Evita duplicatas, adicionando os sufixos _2, _3, ...
        contador = 2
        nome_unico = nome_modificado
        while os.path.exists(caminho_destino):
            nome_unico = f"{nome_modificado}_{contador}"
            caminho_destino = os.path.join(destino, nome_unico)
            contador += 1

        shutil.copytree(caminho_origem, caminho_destino)
        print(f'Pasta {pasta} mudou para {nome_unico}')


FAZENDO AS SUBSTITUIÇÕES CASO NECESSÁRIAS

Substituindo o '_' por espaço, caso precise.

In [ ]:
caminho_raiz = r"D:\Ligia Puc\ProjetoV - A3\Base A3"

for nome in os.listdir(caminho_raiz):
    caminho_antigo = os.path.join(caminho_raiz, nome)
    
    if os.path.isdir(caminho_antigo) and '_' in nome:
        novo_nome = nome.replace('_', ' ')
        caminho_novo = os.path.join(caminho_raiz, novo_nome)
        
        if not os.path.exists(caminho_novo):
            os.rename(caminho_antigo, caminho_novo)
            print(f"Renomeado: {nome} para {novo_nome}")
        else:
            print(f"Já existe: {novo_nome} (não renomeado)")